# Train CNN Classifier on human_ocr_ensembl dataset

The dataset comes from the [Genomic Benchmarks](https://github.com/ML-Bioinfo-CEITEC/genomic_benchmarks). Best reaults achieved are reported in these [tables](https://github.com/ML-Bioinfo-CEITEC/genomic_benchmarks/tree/main/experiments)

In [1]:
!pip install -qq datasets genomic_benchmarks optuna

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 20

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from genomic_benchmarks.data_check import info
import optuna

In [3]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

## Get dataset

In [4]:
info("human_enhancers_cohn", 0)

Dataset `human_enhancers_cohn` has 2 classes: negative, positive.

All lengths of genomic intervals equals 500.

Totally 27791 sequences have been found, 20843 for training and 6948 for testing.


,train,test
negative,10422,3474
positive,10421,3474


In [5]:
dataset = load_dataset("katarinagresova/Genomic_Benchmarks_human_enhancers_cohn")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/477 [00:00<?, ?B/s]

(…)-00000-of-00001-308a8a054210be5f.parquet:   0%|          | 0.00/4.99M [00:00<?, ?B/s]

(…)-00000-of-00001-b9d4d53093c06044.parquet:   0%|          | 0.00/1.66M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20843 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6948 [00:00<?, ? examples/s]

In [6]:
dataset['train'][0]

{'seq': 'TGGTGGTACTTGTCAGGACTTGGAGCAGCAGGTGCAAGATTTAGTGGGTTGGTTTTAGAATATCTGCTTGGAAAGTGGAAAAACTCAATGGATCATCTAGACTTTGGAATTTATCTCCTTCCCCACTTCTCCACTCCCCCAACAACAACAACAACAACAATGACAACAAAAACACCTGGAATAAACAGGTCATACAACGAGGTAGTTGATAGAATAATGTACTTTCCTTTCAGGCACCCCTTGGAGGAGGCAGATTCTGCCCTTTAAGCTGAATCTGCCTTTCCTGCATTTCCTGAAACTCCTGCATTTCCTGAAATCTTCCTGTATTTTCCTGAAATTTCCTGCCATTCCTGAAACTTTAAGGTAACTGTGTCATTAAAGGAAGGAGAGAAGGGAAGTATTAGGACTGCAGATTTGGGGTGCATGATCAGCCTGGCTCTGAGCTTGCAGACTCCCAGAGTCAGGGAAGGGAGGAGCCACCAGCAACCTTGTGGCTTACT',
 'label': 0}

## Encode and split dataset

In [7]:
def one_hot_encode(sequence, max_length=500):
    one_hot = torch.zeros((4, max_length), dtype=torch.float32)

    mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

    for i, nucleotide in enumerate(sequence[:max_length]):
        if nucleotide in mapping:
            one_hot[mapping[nucleotide], i] = 1.0

    return one_hot

class DNADataset(Dataset):
    def __init__(self, data):
        self.dataset = data

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        seq = self.dataset[idx]['seq']
        label = self.dataset[idx]['label']
        encoded_seq = one_hot_encode(seq)
        return encoded_seq, label

In [9]:
ds = dataset["train"].with_format("torch")
ds = DNADataset(ds)

train_size = int(0.8 * len(ds))
val_size = len(ds) - train_size

train_ds, val_ds = torch.utils.data.random_split(ds, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)

val_loader = DataLoader(val_ds, batch_size=16, shuffle=False)

## Define model

In [10]:
# Define a simple CNN for binary classification of DNA sequences
class DNAClassifierCNN(nn.Module):
    def __init__(self, kernel_size=5, out1 = 16, out2 = 32, out3 = 64):
        super(DNAClassifierCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels = 4, out_channels = out1, kernel_size=kernel_size, stride=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=out1, out_channels=out2, kernel_size=kernel_size, stride=1)

        self.relu = nn.LeakyReLU()
        self.fc1 = nn.Linear(self.count_flatten_size(), out3)
        self.fc2 = nn.Linear(out3, 1)
        self.sigmoid = nn.Sigmoid()

    def count_flatten_size(self):
        dummy_input = torch.zeros(1, 4, 500)
        dummy_output = self.pool(self.conv2(self.pool(self.conv1((dummy_input)))))
        return dummy_output.view(-1).size(0)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.reshape(x.size(0), -1)  # Flatten for fully connected layer
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x


In [11]:
# Training loop
def train_model(model, train_loader, optimizer, criterion):
    model.train()
    for batch in train_loader:
        inputs, labels = batch
        labels = labels.float().to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs.to(DEVICE))
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

In [12]:
def evaluate_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for batch in test_loader:
            inputs, labels = batch
            labels = labels.float().to(DEVICE)

            outputs = model(inputs.to(DEVICE))
            loss = criterion(outputs.squeeze(), labels)
            total_loss += loss.item()
            preds = (outputs.squeeze() > 0.5).float()
            correct += (preds == labels).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = correct / len(test_loader.dataset)
    return avg_loss, accuracy

In [13]:
# Run model training and evaluation after each epoch
def evaluation_loop(model, epochs, lr):

    adam = optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    for epoch in range(epochs):
        train_model(model, train_loader, adam, criterion)
        avg_loss, accuracy = evaluate_model(model, val_loader, criterion)
        print(f'Epoch {epoch + 1}/{epochs}, Validation Loss: {avg_loss}, Accuracy: {accuracy}')

    avg_loss, accuracy = evaluate_model(model, val_loader, criterion)

    print(f'Loss: {avg_loss}, Accuracy: {accuracy}\n')

    return accuracy

## Perform training

In [14]:
model = DNAClassifierCNN().to(DEVICE)
evaluation_loop(model, epochs=5, lr=0.001)

Epoch 1/5, Validation Loss: 0.6162696354005529, Accuracy: 0.6632285919884865
Epoch 2/5, Validation Loss: 0.6267942871855593, Accuracy: 0.6440393379707364
Epoch 3/5, Validation Loss: 0.6195128856947596, Accuracy: 0.6670664427920364
Epoch 4/5, Validation Loss: 0.6146098205184571, Accuracy: 0.6646677860398177
Epoch 5/5, Validation Loss: 0.644563128208292, Accuracy: 0.6459582633725114
Loss: 0.644563128208292, Accuracy: 0.6459582633725114



0.6459582633725114

## Hyperparam optimization

Let's try to optimize the learning rate, number of training epochs and size of the convolution kernel

In [15]:
def objective(trial):
    lr = trial.suggest_float('learning_rate', 0.00001, 0.01)
    epochs = trial.suggest_int('epochs', 5, 10)
    kernel_size = trial.suggest_int('kernel_size', 3, 5)
    out1 = trial.suggest_int("out1", 8, 32, step = 8)
    out2 = trial.suggest_int("out2", 32, 64, step = 16)
    out3 = trial.suggest_int("out3", 64, 128, step = 32)

    print(f"LR: {lr}, Epochs: {epochs}, Kernel size: {kernel_size}, out1: {out1}, out2: {out2}, out3: {out3}")

    model = DNAClassifierCNN(kernel_size=kernel_size, out1 = out1, out2 = out2, out3 = out3).to(DEVICE)

    acc = evaluation_loop(model, epochs, lr)
    return acc

In [16]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)

[I 2024-11-20 10:03:37,418] A new study created in memory with name: no-name-fdb091ae-f9c6-41ab-960c-96db7ca4bd50


LR: 0.00876297358566723, Epochs: 7, Kernel size: 5, out1: 24, out2: 64, out3: 128
Epoch 1/7, Validation Loss: 0.6357743537974083, Accuracy: 0.6452386663468458
Epoch 2/7, Validation Loss: 0.6155748355891056, Accuracy: 0.6644279203645959
Epoch 3/7, Validation Loss: 0.6073950420622625, Accuracy: 0.6637083233389302
Epoch 4/7, Validation Loss: 0.6070841420655964, Accuracy: 0.665867114415927
Epoch 5/7, Validation Loss: 0.6180611169429574, Accuracy: 0.6697049652194771
Epoch 6/7, Validation Loss: 0.6256526942682449, Accuracy: 0.6615495322619334
Epoch 7/7, Validation Loss: 0.6662883572994064, Accuracy: 0.6536339649796115


[I 2024-11-20 10:14:33,227] Trial 0 finished with value: 0.6536339649796115 and parameters: {'learning_rate': 0.00876297358566723, 'epochs': 7, 'kernel_size': 5, 'out1': 24, 'out2': 64, 'out3': 128}. Best is trial 0 with value: 0.6536339649796115.


Loss: 0.6662883572994064, Accuracy: 0.6536339649796115

LR: 0.0005368022253535716, Epochs: 8, Kernel size: 3, out1: 16, out2: 48, out3: 64
Epoch 1/8, Validation Loss: 0.6166932215179063, Accuracy: 0.6533940993043895
Epoch 2/8, Validation Loss: 0.6204032910509585, Accuracy: 0.660110338210602
Epoch 3/8, Validation Loss: 0.6489276495473139, Accuracy: 0.6320460542096425
Epoch 4/8, Validation Loss: 0.6080861572333223, Accuracy: 0.6704245622451427
Epoch 5/8, Validation Loss: 0.6500968282250152, Accuracy: 0.646677860398177
Epoch 6/8, Validation Loss: 0.6496010034477117, Accuracy: 0.6543535620052771
Epoch 7/8, Validation Loss: 0.7236730757801012, Accuracy: 0.6538738306548333
Epoch 8/8, Validation Loss: 0.7999401061699308, Accuracy: 0.6435596066202927


[I 2024-11-20 10:26:57,581] Trial 1 finished with value: 0.6435596066202927 and parameters: {'learning_rate': 0.0005368022253535716, 'epochs': 8, 'kernel_size': 3, 'out1': 16, 'out2': 48, 'out3': 64}. Best is trial 0 with value: 0.6536339649796115.


...

Loss: 49.44125158668021, Accuracy: 0.5058767090429359

LR: 0.007671599047248667, Epochs: 8, Kernel size: 3, out1: 8, out2: 64, out3: 64
Epoch 1/8, Validation Loss: 0.6028752918444374, Accuracy: 0.6656272487407052
Epoch 2/8, Validation Loss: 0.6445384883104156, Accuracy: 0.6723434876469178
Epoch 3/8, Validation Loss: 0.6140089877720537, Accuracy: 0.6586711441592709
Epoch 4/8, Validation Loss: 0.6055018223565201, Accuracy: 0.6764212041256896
Epoch 5/8, Validation Loss: 0.6257566624674303, Accuracy: 0.6706644279203646
Epoch 6/8, Validation Loss: 0.6647361653974686, Accuracy: 0.6509954425521708
Epoch 7/8, Validation Loss: 0.6738480527053847, Accuracy: 0.6426001439194051
Epoch 8/8, Validation Loss: 0.7392166599460032, Accuracy: 0.6428400095946271


[I 2024-11-20 10:54:50,541] Trial 3 finished with value: 0.6428400095946271 and parameters: {'learning_rate': 0.007671599047248667, 'epochs': 8, 'kernel_size': 3, 'out1': 8, 'out2': 64, 'out3': 64}. Best is trial 0 with value: 0.6536339649796115.


Loss: 0.7392166599460032, Accuracy: 0.6428400095946271

LR: 0.007808254569311911, Epochs: 9, Kernel size: 5, out1: 16, out2: 64, out3: 64
Epoch 1/9, Validation Loss: 0.6123954980309438, Accuracy: 0.6529143679539458
Epoch 2/9, Validation Loss: 0.6211524053094944, Accuracy: 0.6517150395778364
Epoch 3/9, Validation Loss: 0.6017581336342969, Accuracy: 0.6586711441592709
Epoch 4/9, Validation Loss: 0.594432766295941, Accuracy: 0.6651475173902615
Epoch 5/9, Validation Loss: 0.5974783655327399, Accuracy: 0.6790597265531303
Epoch 6/9, Validation Loss: 0.5942470773883249, Accuracy: 0.6745022787239147
Epoch 7/9, Validation Loss: 0.6123029618770227, Accuracy: 0.6687455025185896
Epoch 8/9, Validation Loss: 0.6158194443731929, Accuracy: 0.6668265771168146
Epoch 9/9, Validation Loss: 0.6281090315388537, Accuracy: 0.656272487407052


[I 2024-11-20 11:08:47,513] Trial 4 finished with value: 0.656272487407052 and parameters: {'learning_rate': 0.007808254569311911, 'epochs': 9, 'kernel_size': 5, 'out1': 16, 'out2': 64, 'out3': 64}. Best is trial 4 with value: 0.656272487407052.


Loss: 0.6281090315388537, Accuracy: 0.656272487407052



In [17]:
print(f"Best hyperparameters: {study.best_params}")
print(f"Best value (validation AU PRC): {study.best_value}")

Best hyperparameters: {'learning_rate': 0.007808254569311911, 'epochs': 9, 'kernel_size': 5, 'out1': 16, 'out2': 64, 'out3': 64}
Best value (validation AU PRC): 0.656272487407052


In [18]:
test_ds = DNADataset(dataset["test"].with_format("torch"))
test_loader = DataLoader(test_ds, batch_size=16, shuffle=False)
# Use the best hyperparameters from the Optuna study
best_params = study.best_params
best_model = DNAClassifierCNN(kernel_size=best_params['kernel_size'], out1 = best_params['out1'], out2 = best_params['out2'], out3 = best_params['out3']).to(DEVICE)
# Train the best model with the best hyperparameters
evaluation_loop(best_model, epochs=best_params['epochs'], lr=best_params['learning_rate'])
# Compute test set performance
criterion = nn.BCELoss()
test_loss, test_accuracy = evaluate_model(best_model, test_loader, criterion)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

Epoch 1/9, Validation Loss: 0.61503400804896, Accuracy: 0.6704245622451427
Epoch 2/9, Validation Loss: 0.6215493655067751, Accuracy: 0.6270088750299833
Epoch 3/9, Validation Loss: 0.590600868066152, Accuracy: 0.6819381146557928
Epoch 4/9, Validation Loss: 0.5969986766005841, Accuracy: 0.6771408011513552
Epoch 5/9, Validation Loss: 0.5894420776102278, Accuracy: 0.6670664427920364
Epoch 6/9, Validation Loss: 0.6178522280121215, Accuracy: 0.6593907411849365
Epoch 7/9, Validation Loss: 0.6085663727873587, Accuracy: 0.6816982489805709
Epoch 8/9, Validation Loss: 0.6203192296612765, Accuracy: 0.6711441592708084
Epoch 9/9, Validation Loss: 0.6409268842803107, Accuracy: 0.6665867114415928
Loss: 0.6409268842803107, Accuracy: 0.6665867114415928

Test Loss: 0.6248679196697542, Test Accuracy: 0.6689694876223373
